In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

colunas = ['Nome', 'Unidades', 'Investimento', 'Retorno', 'Faturamento', 'Contato', 'Referência']
df = pd.DataFrame(columns=colunas)

url = 'https://franquias.portaldofranchising.com.br/franquias-de-alimentacao/?_gl=1*1dldi9q*_ga*NjI3MzYyODA4LjE2OTk5NTkzMTM.*_ga_TPGK1V9KED*MTcwMDM1MTM1Ni40LjEuMTcwMDM1MTM2Mi41NC4wLjA.'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
filtro1 = soup.find_all('a')
for e in filtro1:
    if e.get('title'):
        if not ('#' in e.get('href')):
            nome = e.get('title')
            url_franquia = e.get('href')
            page_franquia = requests.get(url_franquia)
            soup = BeautifulSoup(page_franquia.text, 'html.parser')

            achou_unidades = False
            achou_investimento = False
            achou_retorno = False
            achou_contato = False
            achou_faturamento = False

            faturamento = "Não informado"

            for e in soup.find_all('strong'):
                conteudo = e.get_text()
                separacao_string = conteudo.split()
                if not achou_investimento and separacao_string[0] == 'a':
                    investimento = ' '.join(separacao_string)
                    achou_investimento = True
                elif not achou_retorno and separacao_string[0] == 'de':
                    retorno = ' '.join(separacao_string)
                    achou_retorno = True
                elif not achou_faturamento and separacao_string[0][0] == 'R':
                    faturamento = ' '.join(separacao_string)
                    achou_faturamento = True
                elif not achou_contato and len(separacao_string)==2:
                    contato = ' '.join(separacao_string)
                    achou_contato = True
                elif not achou_unidades:
                    n_unidades = int(' '.join(separacao_string))
                    achou_unidades = True
            
            dados = {
                'Nome':nome,
                'Unidades':n_unidades,
                'Investimento':investimento,
                'Retorno':retorno,
                'Faturamento':faturamento,
                'Contato':contato,
                'Referência':url_franquia
            }
            df = df._append(dados, ignore_index=True)

# Ordena por franquias de alimentação com maior quantidade de unidades
df_ordenado = df.sort_values(by="Unidades", ascending=False)
maior_unidade = df_ordenado['Unidades'].iloc[0]
referencia = df.loc[df['Unidades'] == maior_unidade, 'Referência'].values[0]

print(f'Franquia com maior quantidade de unidades {df_ordenado["Nome"].iloc[0]}; Referência {referencia}')

display(df_ordenado.head())



Franquia com maior quantidade de unidades CACAU SHOW; Referência https://franquias.portaldofranchising.com.br/franquia-cacau-show-valor/


,Nome,Unidades,Investimento,Retorno,Faturamento,Contato,Referência
19,CACAU SHOW,3868,a partir de R$64.900,de 14 a 24 meses,Não informado,(11) 3526-9854,https://franquias.portaldofranchising.com.br/f...
7,SUBWAY,1574,a partir de R$490.000,de 36 a 60 meses,Não informado,(11) 3077-1010,https://franquias.portaldofranchising.com.br/f...
49,OGGI SORVETES,1003,a partir de R$50.000,de 18 a 24 meses,Não informado,(11) 4398-6830,https://franquias.portaldofranchising.com.br/f...
56,BOB´S,955,a partir de R$1.022.461,de 18 a 48 meses,R$180.000,(11) 96624-6715,https://franquias.portaldofranchising.com.br/f...
14,CHIQUINHO SORVETES,725,a partir de R$315.000,de 24 a 36 meses,R$60.000 a R$70.000,(17) 3211-8200,https://franquias.portaldofranchising.com.br/f...
